In [1]:
import pandas as pd
import numpy as np

from datascience import *

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

Preprocessing in Excel:
    1. Add building names in row 0
    2. Remove rows 1-3
    3. Remove average, min, max rows at eof
    4. Export as csv

In [2]:
raw = pd.read_csv('rawdata.csv',index_col=0,na_values=['#########'])

raw.index = pd.to_datetime(raw.index,format='%a %m/%d/%y %H:00')

raw.head(2)

,YUAG,Unnamed: 2,Berkeley,Unnamed: 4,Hopper,Unnamed: 6,304Elm,Unnamed: 8,Davenport,Unnamed: 10,...,53Wall,Unnamed: 16,Sprague,Unnamed: 18,Malone,Unnamed: 20,Trumbull,Unnamed: 22,17HH,Unnamed: 24
2018-01-01 00:00:00,NaN,7929946.5,NaN,7241231.2,NaN,2766963.5,NaN,1109119.8,NaN,4306847.8,...,NaN,3024576.8,NaN,358345.5,NaN,6543549.3,NaN,4441711.2,NaN,8929603.5
2018-01-01 01:00:00,157.1,7930103.6,69.6,7241300.8,87.6,2767051.1,4.6,1109124.4,95.0,4306942.8,...,-530770.5,2493806.4,34.0,358379.5,111.3,6543660.7,54.6,4441765.7,-1510122.1,7419481.4


This cell isolates the YUAG data, renames the columns, interpolates missing values, and adds an 'Hourly' column that gives the hourly consumption.

As it stands, I cannot figure out how to smooth and interpolate the demand data, so I am ignoring it for now.

In [5]:
yuag = raw[['YUAG','Unnamed: 2']]
yuag.columns = ['Demand','Consumption']

yuag.loc[:,'Consumption'] = yuag['Consumption'].replace(0, np.NaN)

# These lines are a work-in-progress to set NaN to the incorrect built-in demand data
# yuag.loc[:,'Demand'] = yuag['Demand'].clip(0,5000)
# yuag['Demand'].where(yuag['Demand']<0, np.NaN)



The cell below interpolates both demand and consumption data, and creates an Hourly Consumption column

In [6]:
yuag = yuag.interpolate(method='time',limit=3)

yuag = yuag.assign(Hourly=yuag['Consumption'].diff())

yuag['Hourly'].describe()

count    4990.000000
mean      192.996192
std       125.063076
min         0.000000
25%       120.700000
50%       148.350000
75%       283.000000
max      3464.300000
Name: Hourly, dtype: float64